In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163762 sha256=3904f0676f93e96de0a4d5cf87200d2dba81f0364c96b0ed3be41e6f53b0d01c
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
MOVIES_FILE = 'movies_small.csv'
RATINGS_FILE = 'ratings_small.csv'

In [4]:
import pandas as pd

# Loading datasets
df_movies = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/ML - CS576/RecSys/{MOVIES_FILE}')
df_ratings = pd.read_csv(f'/content/drive/MyDrive/Colab Notebooks/ML - CS576/RecSys/{RATINGS_FILE}')

In [5]:
df_ratings = df_ratings.rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
df_ratings = df_ratings.drop('timestamp', axis=1)

In [6]:
df2 = pd.DataFrame([
    [ 1000, 8368, 5.0 ],
    [ 1000, 69844, 5.0 ],
    [ 1000, 88140, 5.0 ],
    [ 1000, 122892, 5.0 ],
    [ 1000, 130842, 5.0 ],
    [ 1000, 60979, 5.0 ],
    [ 1000, 136864, 5.0 ],
    [ 1000, 2640, 5.0 ],
    [ 1000, 2641, 5.0 ],
    [ 1000, 46530, 5.0 ],
    [ 1000, 1387, 0.5 ],
    [ 1000, 50, 2.0 ],
    [ 1000, 356, 1.0 ],
    [ 1000, 593, 2.0 ],
    [ 1000, 2028, 1.0 ],
    [ 1000, 1219, 2.0 ],
    [ 1000, 2389, 0.5 ],
    [ 1000, 260, 0.5 ],
    [ 1000, 329, 0.5 ],
    [ 1000, 33493, 0.5 ],
    [ 1000, 2019, 0.5 ]
  ],
  columns=['user_id', 'item_id', 'rating'])
df_ratings = pd.concat([df_ratings, df2])

In [7]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader

reader = Reader()
# data = Dataset.load_builtin('ml-100k')
data = Dataset.load_from_df(df_ratings[['user_id', 'item_id', 'rating']], reader)

In [27]:
from surprise import KNNWithMeans, KNNBasic
from surprise.model_selection import GridSearchCV
from surprise.model_selection import train_test_split


# sim_options = {
#     "name": ["msd", "cosine"],
#     "min_support": [3, 4, 5],
#     "user_based": [False, True],
# }

sim_options = {
    "name": ["cosine"],
    "min_support": [True],
    "user_based": [True],
    "max_k": [3, 5, 7, 10],
    "min_k": [2]
}

param_grid = {"sim_options": sim_options}

gs = GridSearchCV(KNNBasic, param_grid, measures=["rmse", "mae"], cv=3)
gs.fit(data)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [28]:
print("GS Best RMSE: ", gs.best_score["rmse"])
print("GS Best MAE: ", gs.best_score["mae"])

GS Best RMSE:  0.9802661690086362
GS Best MAE:  0.7553643170253813


In [29]:
algo = gs.best_estimator["rmse"]
trainset = data.build_full_trainset()
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [30]:
gs.best_params['rmse']

{'sim_options': {'name': 'cosine',
  'min_support': True,
  'user_based': True,
  'max_k': 3,
  'min_k': 2}}

In [31]:
gs.best_params['mae']

{'sim_options': {'name': 'cosine',
  'min_support': True,
  'user_based': True,
  'max_k': 3,
  'min_k': 2}}

In [32]:
# testset = trainset.build_anti_testset()

# predictions = algo.test(testset)

In [33]:
def recommend_n_movies(ratings_data, movies_data, user_id, n=10, verbose=True):
    all_movies = ratings_data['item_id'].unique()
    user_movies = ratings_data[ratings_data['user_id'] == user_id]['item_id'].unique()
    new_movies = list(set(all_movies) - set(user_movies))

    if verbose:
      print(f'Movies user {user_id} has already seen: -\n')
      for movie_id in user_movies:
        print(movies_data[movies_data.movieId == movie_id].title.values[0])

    # Predict ratings for new movies
    predictions = [algo.predict(user_id, movie_id) for movie_id in new_movies]

    # Sort the predictions by estimated rating
    predictions.sort(key=lambda x: x.est, reverse=True)

    # Get top n recommendations
    n_recommendations = [prediction.iid for prediction in predictions[:n]]

    if verbose:
      print()
      print('Recommendations: -\n')
      for movie_id in n_recommendations:
        print(movies_data[movies_data.movieId == movie_id].title.values[0])


    return n_recommendations

In [36]:
result = recommend_n_movies(df_ratings, df_movies, 272)

Movies user 272 has already seen: -

Shawshank Redemption, The (1994)
Terminator 2: Judgment Day (1991)
Terminator, The (1984)
Fifth Element, The (1997)
Men in Black (a.k.a. MIB) (1997)
Truman Show, The (1998)
Titanic (1997)
Blade (1998)
Matrix, The (1999)
Green Mile, The (1999)
Lord of the Rings: The Fellowship of the Ring, The (2001)
Equilibrium (2002)
Matrix Reloaded, The (2003)
Serenity (2005)
V for Vendetta (2006)
Casino Royale (2006)
Ratatouille (2007)
300 (2007)
Star Trek (2009)
Limitless (2011)
Avengers, The (2012)
Django Unchained (2012)
Divergent (2014)
Interstellar (2014)
22 Jump Street (2014)
Kingsman: The Secret Service (2015)
Doctor Strange (2016)
Inside Out (2015)
Big Short, The (2015)
The Nice Guys (2016)
Coco (2017)

Recommendations: -

Saving Santa (2013)
Tom and Jerry: A Nutcracker Tale (2007)
Wings, Legs and Tails (1986)
Heidi Fleiss: Hollywood Madam (1995)
Awfully Big Adventure, An (1995)
What Men Talk About (2010)
What Happened Was... (1994)
The Girl with All the 